In [1]:
%pylab inline

import numpy as np
from __future__ import division

import matplotlib.pyplot as plt
import seaborn as sns

import logging
logger = logging.getLogger()

Populating the interactive namespace from numpy and matplotlib


第1章 色彩、对比度和通道
===========


### 1.1 难以取舍
色彩 VS 对比度


### 1.2 让色彩更自然
人眼与相机的区别：

1. 色偏：物体受环境光影响而产生不自然的色调时，
   + 人眼自动纠正（人总是把环境光的颜色看成是中性灰）。   
   + 相机忠实记录。   
   
   冷色色偏很少能让人接受，而暧色色偏有时是讨喜的。    
   
   补充：显示器修色时，不会注意到色偏；制成印刷品时，才会发现色偏。
      
2. 同时对比效应（存在背景色时）
   + 人眼会夸大前景色和背景色的差别（视觉错觉迫使前景色朝着与背景色相反的方向发展）。    
   + 相机拍摄的照片常常充满**近似色**，缺乏对比。   
   
3. 色适应：
   + 人眼会快速适应不同的灯光强度。    
   + 人眼会自动减少反射光和闪烁光（相机常见到人物脸上满是反光）。 $\implies$ 室内肖像摄影时化浓妆。

4. 其他人眼视觉错觉：
   + 人眼注视一个对象时，它的对比度增强，周围次要景物的对比度降低。
   + 人对自己不感兴趣的背景的色彩感知能力会减弱。
   + 对象的色彩越鲜艳，它与背景的对比就越鲜明。
   + 人眼看来，景物最暗的部分（即使是明快色彩的一部分）通常是无色的。    
     例子：绿色大衣的最暗皱褶，相机记录是深绿色。
 
 
迎合人类视错觉：    
减少色偏，抑制反光，让主体对象的色彩更丰富，让次要背景变灰。


### 1.4 军火库

本书只关注几个造成原稿和修正图间差异的问题：

1. 曲线。   
   确保完整的阶调范围，突出最重要区域的细节，避免颜色失真。
   
2. 通道混合。
   解决曲线不能解决的某些问题，最典型的用法是修补对比度弱的通道。
   
3. 锐化。

4. 高光和阴影。
   数码相机（不同于底片）为了加强中间调的对比度，会牺牲高光和阴影。高光和阴影变成极端的颜色，失去层次感。     
   “阴影/高光“命令获得最佳效果 $\gets$ 领会锐化滤镜的基本工作原理。

5. 滥用。
   1. 选区和蒙版。   
      合理使用场景：
      + 剪下局部，或者扩展背景。
      + 一张照片中有两处冲突光源。
      + 色彩置换（蓝衬衫 $\to$ 红衬衫）。

我们总是希望使用曲线可以解决图面问题，除非它确实不够用了，再补充其他命令。

### 1.6 通道测试

一束花的 10 个通道黑白图片，请推测各个通道的归属（RGB - 3, LAB - 3, CMYK - 4)。